In [1]:
import json

# Predefined dictionary to map Python data types to X++ data types
data_type_dict = {
    "str": "str",
    "int": "int",
    "float": "real",
}

def get_xpp_data_type(python_data_type):
        # Use the predefined data_type_dict to get the X++ data type
        return data_type_dict.get(python_data_type, python_data_type)

def generate_xpp_data_contract_class(class_name, json_obj, indent="    "):
    json_obj = json.loads(json_obj)
    xpp_code = f"[DataContract]\n"
    xpp_code += f"class {class_name}\n{{\n"

    def process_object(obj, indent):
        nonlocal xpp_code  # Use nonlocal to access the xpp_code variable from the outer scope.
        for key, value in obj.items():
            if isinstance(value, dict):
                nested_class_name = f"{class_name}_{key.capitalize()}"
                xpp_code += f"{indent}private {nested_class_name} {key};\n"
                xpp_code += f"{indent}[DataMemberAttribute('{key}')]\n"
                xpp_code += f"{indent}public {nested_class_name} parm{key[0].upper()}{key[1:]}({nested_class_name} _{key} = {key})\n"
                xpp_code += f"{indent}{{\n"
                xpp_code += f"{indent}    {key} = _{key};\n"
                xpp_code += f"{indent}    return {key};\n"
                xpp_code += f"{indent}}}\n\n"
                xpp_code += f"\n{indent}[DataContract]\n"
                xpp_code += f"{indent}class {nested_class_name}\n{indent}{{\n"
                process_object(value, indent + "    ")
                xpp_code += f"{indent}}}\n\n"
            elif isinstance(value, list):
                nested_class_name = f"{class_name}_{key.capitalize()}"
                xpp_code += f"{indent}private List {key};\n"
                xpp_code += f"{indent}[DataMemberAttribute('{key}'), DataCollectionAttribute(Types::Class, classStr({nested_class_name}))];\n"
                xpp_code += f"{indent}public List parm{key[0].upper()}{key[1:]}(List _{key} = {key})\n"
                xpp_code += f"{indent}{{\n"
                xpp_code += f"{indent}    {key} = _{key};\n"
                xpp_code += f"{indent}    return {key};\n"
                xpp_code += f"{indent}}}\n\n"
                if len(value) > 0 and isinstance(value[0], dict):
                    xpp_code += f"\n{indent}[DataContract]\n"
                    xpp_code += f"{indent}class {nested_class_name}\n{indent}{{\n"
                    process_object(value[0], indent + "    ")
                    xpp_code += f"{indent}}}\n\n"
            else:
                python_data_type = type(value).__name__
                xpp_data_type = get_xpp_data_type(python_data_type)
                xpp_code += f"{indent}private {xpp_data_type} {key};\n"
                xpp_code += f"{indent}[DataMemberAttribute('{key}')]\n"
                xpp_code += f"{indent}public {xpp_data_type} parm{key[0].upper()}{key[1:]}({xpp_data_type} _{key} = {key})\n"
                xpp_code += f"{indent}{{\n"
                xpp_code += f"{indent}    {key} = _{key};\n"
                xpp_code += f"{indent}    return {key};\n"
                xpp_code += f"{indent}}}\n\n"

    process_object(json_obj, "    ")
    xpp_code += "}\n"

    return xpp_code


# Example usage:
json_data = '''
{
  "SalesNumber" : "S0001",
  "Account":"UK00BL1",
  "Name":"Bulb Inc. ELSON",
  "DeliveryDate":"19/07/2021",
  "DeliveryAddress":
  {
    "AddressName":"Blub Inc UK",
    "City":"ELSON",
    "ISOCountry":"GB",
    "PostCode":"",
    "Street":"85 Whitchurch Road"
  },
  "Lines":[
    {
       "LineID":1,
       "ItemName":"Bulb 100W",     
       "ItemCode":"BL00100W",   
       "Price":1.5,
       "CurrencyISO":"GBP",
       "Qty":3,
       "UnitID":"ea"
    },
    {
       "LineID":2,
       "ItemName":"Bulb LED 10W",     
       "ItemCode":"LED0010W",   
       "Price":15.45,
       "CurrencyISO":"GBP",
       "Qty":2,       
       "UnitID":"box"
    }
  ]
}
'''

class_name = "SOHeader"

xpp_code = generate_xpp_data_contract_class(class_name, json_data)
print(xpp_code)


[DataContract]
class SOHeader
{
    private str SalesNumber;
    [DataMemberAttribute('SalesNumber')]
    public str parmSalesNumber(str _SalesNumber = SalesNumber)
    {
        SalesNumber = _SalesNumber;
        return SalesNumber;
    }

    private str Account;
    [DataMemberAttribute('Account')]
    public str parmAccount(str _Account = Account)
    {
        Account = _Account;
        return Account;
    }

    private str Name;
    [DataMemberAttribute('Name')]
    public str parmName(str _Name = Name)
    {
        Name = _Name;
        return Name;
    }

    private str DeliveryDate;
    [DataMemberAttribute('DeliveryDate')]
    public str parmDeliveryDate(str _DeliveryDate = DeliveryDate)
    {
        DeliveryDate = _DeliveryDate;
        return DeliveryDate;
    }

    private SOHeader_Deliveryaddress DeliveryAddress;
    [DataMemberAttribute('DeliveryAddress')]
    public SOHeader_Deliveryaddress parmDeliveryAddress(SOHeader_Deliveryaddress _DeliveryAddress = DeliveryA